# Deep Learning LSTM Model

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
data = pd.read_csv("tweetdata",encoding= 'unicode_escape')

In [3]:
def getTextAnalysis(a):
    if a > 0:
        return "1"
    else:
        return "0"

In [4]:
data['sentiment'] = data['Polarity'].apply(getTextAnalysis)

In [5]:
data = data[['Data','sentiment']]

In [6]:
print(data[ data['sentiment'] == '1'].size)
print(data[ data['sentiment'] == '0'].size)

9892
10108


In [7]:
data

,Data,sentiment
0,cnsnews benshapiro Can COVID19Vaccine end the ...,0
1,IntrepidWarrior realDonaldTrump Can COVID19Vac...,0
2,edyong209 HelenBranswell Can COVID19Vaccine en...,0
3,AdamTexDavis Can COVID19Vaccine end the pandemic,0
4,HegKong Can COVID19Vaccine end the pandemic,0
...,...,...
9995,UAlberta is happy to have you on campus This f...,1
9996,RT WilsonMHI NEW from MHI Vaccines when approp...,1
9997,BREAKING Clinical trial for AstraZenecaOxford ...,0
9998,Supply chain professionals point to the many c...,1


In [8]:
for ids,r in data.iterrows():
    r[1] = r[1].replace('rt',' ')
    
max_fatures = 2000
token = Tokenizer(num_words=max_fatures, split=' ')
token.fit_on_texts(data['Data'].values)
X = token.texts_to_sequences(data['Data'].values)
feature = pad_sequences(X)

In [9]:
embed = 128
lstm = 196

model = Sequential()
model.add(Embedding(max_fatures, embed,input_length = feature.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 26, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
target = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(feature,target, test_size = 0.40, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6000, 26) (6000, 2)
(4000, 26) (4000, 2)


In [11]:
b_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=b_size)


Epoch 1/20
188/188 [==============================] - 30s 157ms/step - loss: 0.2585 - accuracy: 0.8815
Epoch 2/20
188/188 [==============================] - 37s 197ms/step - loss: 0.0271 - accuracy: 0.9913
Epoch 3/20
188/188 [==============================] - 50s 266ms/step - loss: 0.0136 - accuracy: 0.9963
Epoch 4/20
188/188 [==============================] - 38s 204ms/step - loss: 0.0022 - accuracy: 0.9995
Epoch 5/20
188/188 [==============================] - 34s 183ms/step - loss: 0.0056 - accuracy: 0.9988
Epoch 6/20
188/188 [==============================] - 66s 353ms/step - loss: 0.0120 - accuracy: 0.9955
Epoch 7/20
188/188 [==============================] - 47s 248ms/step - loss: 0.0075 - accuracy: 0.9983
Epoch 8/20
188/188 [==============================] - 52s 274ms/step - loss: 0.0057 - accuracy: 0.9987
Epoch 9/20
188/188 [==============================] - 34s 182ms/step - loss: 6.8569e-04 - accuracy: 1.0000
Epoch 10/20
188/188 [==============================] - 36s 191ms/step

In [12]:
validation_size = 1500

X_val = X_test[-validation_size:]
Y_val = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,accuacy = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("Score: %.2f" % (score))
print("Accuacy: %.2f" % (accuacy))

79/79 - 2s - loss: 0.0101 - accuracy: 0.9980
Score: 0.01
Accuacy: 1.00


In [15]:
postive_count, negative_count, postive_correct, negative_correct = 0, 0, 0, 0
for x in range(len(X_val)):
    result = model.predict(X_val[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_val[x]):
        if np.argmax(Y_val[x]) == 0:
            negative_correct += 1
        else:
            postive_correct += 1
    if np.argmax(Y_val[x]) == 0:
        negative_count += 1
    else:
        postive_count += 1
        
print("Positive Accuacy", postive_correct/postive_count*100, "%")
print("Negative Accuacy", negative_correct/negative_count*100, "%")

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1